In [7]:
import numpy as np
import pandas as pd
from scipy.stats import kstest, shapiro, ttest_rel

from package.idw import holdout_idw, kfoldcv_idw
from package.kriging import holdout_kriging, kfoldcv_kriging


readings = pd.read_csv("merged_data.csv")
boundary = pd.read_csv("boundary_tres.csv")

rmse_results1 = holdout_idw(readings, boundary, 100, 2, 0.8)
rmse_results2 = holdout_kriging(readings, boundary, 100, 100, 0.8)
rmse_results3 = kfoldcv_idw(readings, boundary, 100, 2, 6)
rmse_results4 = kfoldcv_kriging(readings, boundary, 100, 10, 6)
rmse_results5 = kfoldcv_idw(readings, boundary, 100, 2, 18)
rmse_results6 = kfoldcv_kriging(readings, boundary, 100, 10, 18)

dataframes = [pd.DataFrame(list(results.items()), columns=['timestamp', 'rmse']) for results in [rmse_results1, rmse_results2, rmse_results3, rmse_results4, rmse_results5, rmse_results6]]
titles = ['Holdout IDW', 'Holdout Kriging', 'K-fold IDW', 'K-fold Kriging', 'LOOCV IDW', 'LOOCV Kriging']

1
2
3
4
5
6


In [40]:
summary_stats = {}

for df, title in zip(dataframes, titles):
    summary_stats[title] = {
        'mean': df['rmse'].mean(),
        'median': df['rmse'].median(),
        'min': df['rmse'].min(),
        'max': df['rmse'].max(),
        'std': df['rmse'].std()
    }

methods_results = pd.DataFrame(summary_stats).T
print(methods_results)

latex_table = methods_results.to_latex(index=True, column_format='|c|c|c|c|c|c|', bold_rows=True, float_format="%.2f", escape=False)

with open('methods_results.tex', 'w') as f:
    f.write(latex_table)

                       mean     median        min         max        std
Holdout IDW       90.130219  69.181322  10.959524  233.912695  67.039400
Holdout Kriging   97.427703  76.828372  14.492401  245.099148  71.577935
K-fold IDW       102.597679  93.496059  44.018456  191.192103  43.462948
K-fold Kriging   107.284302  96.774923  41.654320  200.765702  46.683608
LOOCV IDW         83.576592  75.040236  31.232982  163.276827  40.576969
LOOCV Kriging     89.060384  80.082074  29.395128  175.293571  44.068661


In [41]:
df_list = []

for df, method in zip(dataframes, titles):
    df['method'] = method 
    df_list.append(df) 

combined_df = pd.concat(df_list, ignore_index=True)

combined_df[['Validation Technique', 'Interpolation Method']] = combined_df['method'].str.split(' ', n=1, expand=True)

combined_df.drop(columns=['method'], inplace=True)
combined_df.rename(columns={'rmse': 'RMSE'}, inplace=True)

print(combined_df)

                      timestamp        RMSE Validation Technique  \
0     2023-07-15 00:00:00+00:00  172.559049              Holdout   
1     2023-07-15 01:00:00+00:00  171.463392              Holdout   
2     2023-07-15 02:00:00+00:00  171.117183              Holdout   
3     2023-07-15 03:00:00+00:00  172.352945              Holdout   
4     2023-07-15 04:00:00+00:00  171.070100              Holdout   
...                         ...         ...                  ...   
2443  2023-07-31 19:00:00+00:00  171.664969                LOOCV   
2444  2023-07-31 20:00:00+00:00  170.169762                LOOCV   
2445  2023-07-31 21:00:00+00:00  172.285392                LOOCV   
2446  2023-07-31 22:00:00+00:00  168.934264                LOOCV   
2447  2023-07-31 23:00:00+00:00  175.293571                LOOCV   

     Interpolation Method  
0                     IDW  
1                     IDW  
2                     IDW  
3                     IDW  
4                     IDW  
...            

In [33]:
r1 = pd.DataFrame(list(rmse_results1.items()), columns=['timestamp', 'rmse'])
r2 = pd.DataFrame(list(rmse_results2.items()), columns=['timestamp', 'rmse'])

r1.rename(columns={'rmse': 'RMSE_IDW'}, inplace=True)
r2.rename(columns={'rmse': 'RMSE_Kriging'}, inplace=True)

combined_rmse_df = pd.merge(r1[['timestamp', 'RMSE_IDW']], 
                             r2[['timestamp', 'RMSE_Kriging']], 
                             on='timestamp')

sampled_df = combined_rmse_df.sample(n=50, random_state=0)
sample_differences = sampled_df['RMSE_IDW'] - sampled_df['RMSE_Kriging']
full_differences = combined_rmse_df['RMSE_IDW'] - combined_rmse_df['RMSE_Kriging']

shapiro_test_result = shapiro(sample_differences)
shapiro_statistic = shapiro_test_result.statistic
shapiro_pvalue = shapiro_test_result.pvalue

print(f'Shapiro-Wilk Test Statistic: {shapiro_statistic}')
print(f'Shapiro-Wilk Test p-value: {shapiro_pvalue}')

if shapiro_pvalue > 0.05:
    print("Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.")
else:
    print("Rifiutiamo l'ipotesi nulla: i dati non seguono una distribuzione normale.")

Shapiro-Wilk Test Statistic: 0.9599182601834915
Shapiro-Wilk Test p-value: 0.08819467329465562
Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.


In [23]:
ks_statistic, p_value_ks = kstest(full_differences, 'norm', args=(full_differences.mean(), full_differences.std()))

print(f"Statistiche K-S: {ks_statistic}")
print(f"p-value del test K-S: {p_value_ks}")

if p_value_ks > 0.05:
    print("Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.")
else:
    print("Rifiutiamo l'ipotesi nulla: i dati non seguono una distribuzione normale.")

Statistiche K-S: 0.07698989004150136
p-value del test K-S: 0.015013541281027802
Rifiutiamo l'ipotesi nulla: i dati non seguono una distribuzione normale.


In [25]:
t_stat, p_value = ttest_rel(full_differences, np.zeros(len(full_differences)))

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Rifiutiamo l'ipotesi nulla: C'è evidenza statistica di una differenza significativa nelle prestazioni tra i due metodi di interpolazione.")
else:
    print("Non si può rifiutare l'ipotesi nulla: on ci sono evidenze statistiche sufficienti per affermare che ci sia una differenza nelle prestazioni tra i due metodi di interpolazione")

T-statistic: -17.9226
P-value: 0.0000
Rifiutiamo l'ipotesi nulla: C'è evidenza statistica di una differenza significativa nelle prestazioni tra i due metodi di interpolazione.


In [34]:
r3 = pd.DataFrame(list(rmse_results3.items()), columns=['timestamp', 'rmse'])
r4 = pd.DataFrame(list(rmse_results4.items()), columns=['timestamp', 'rmse'])

r3.rename(columns={'rmse': 'RMSE_IDW'}, inplace=True)
r4.rename(columns={'rmse': 'RMSE_Kriging'}, inplace=True)

combined_rmse_df = pd.merge(r3[['timestamp', 'RMSE_IDW']], 
                             r4[['timestamp', 'RMSE_Kriging']], 
                             on='timestamp')

sampled_df = combined_rmse_df.sample(n=50, random_state=0)
sample_differences = sampled_df['RMSE_IDW'] - sampled_df['RMSE_Kriging']
full_differences = combined_rmse_df['RMSE_IDW'] - combined_rmse_df['RMSE_Kriging']

shapiro_test_result = shapiro(sample_differences)
shapiro_statistic = shapiro_test_result.statistic
shapiro_pvalue = shapiro_test_result.pvalue

print(f'Shapiro-Wilk Test Statistic: {shapiro_statistic}')
print(f'Shapiro-Wilk Test p-value: {shapiro_pvalue}')

if shapiro_pvalue > 0.05:
    print("Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.")
else:
    print("Rifiutiamo l'ipotesi nulla: i dati non seguono una distribuzione normale.")

Shapiro-Wilk Test Statistic: 0.958089256748092
Shapiro-Wilk Test p-value: 0.07393208172498472
Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.


In [35]:
ks_statistic, p_value_ks = kstest(full_differences, 'norm', args=(full_differences.mean(), full_differences.std()))

print(f"Statistiche K-S: {ks_statistic}")
print(f"p-value del test K-S: {p_value_ks}")

if p_value_ks > 0.05:
    print("Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.")
else:
    print("Rifiutiamo l'ipotesi nulla: i dati non seguono una distribuzione normale.")

Statistiche K-S: 0.05849930778961793
p-value del test K-S: 0.11775180454990986
Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.


In [36]:
t_stat, p_value = ttest_rel(full_differences, np.zeros(len(full_differences)))

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Rifiutiamo l'ipotesi nulla: C'è evidenza statistica di una differenza significativa nelle prestazioni tra i due metodi di interpolazione.")
else:
    print("Non si può rifiutare l'ipotesi nulla: on ci sono evidenze statistiche sufficienti per affermare che ci sia una differenza nelle prestazioni tra i due metodi di interpolazione")

T-statistic: -9.8811
P-value: 0.0000
Rifiutiamo l'ipotesi nulla: C'è evidenza statistica di una differenza significativa nelle prestazioni tra i due metodi di interpolazione.


In [37]:
r5 = pd.DataFrame(list(rmse_results5.items()), columns=['timestamp', 'rmse'])
r6 = pd.DataFrame(list(rmse_results6.items()), columns=['timestamp', 'rmse'])

r5.rename(columns={'rmse': 'RMSE_IDW'}, inplace=True)
r6.rename(columns={'rmse': 'RMSE_Kriging'}, inplace=True)

combined_rmse_df = pd.merge(r5[['timestamp', 'RMSE_IDW']], 
                             r6[['timestamp', 'RMSE_Kriging']], 
                             on='timestamp')

sampled_df = combined_rmse_df.sample(n=50, random_state=0)
sample_differences = sampled_df['RMSE_IDW'] - sampled_df['RMSE_Kriging']
full_differences = combined_rmse_df['RMSE_IDW'] - combined_rmse_df['RMSE_Kriging']

shapiro_test_result = shapiro(sample_differences)
shapiro_statistic = shapiro_test_result.statistic
shapiro_pvalue = shapiro_test_result.pvalue

print(f'Shapiro-Wilk Test Statistic: {shapiro_statistic}')
print(f'Shapiro-Wilk Test p-value: {shapiro_pvalue}')

if shapiro_pvalue > 0.05:
    print("Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.")
else:
    print("Rifiutiamo l'ipotesi nulla: i dati non seguono una distribuzione normale.")

Shapiro-Wilk Test Statistic: 0.9688533091133603
Shapiro-Wilk Test p-value: 0.20779980040587975
Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.


In [38]:
ks_statistic, p_value_ks = kstest(full_differences, 'norm', args=(full_differences.mean(), full_differences.std()))

print(f"Statistiche K-S: {ks_statistic}")
print(f"p-value del test K-S: {p_value_ks}")

if p_value_ks > 0.05:
    print("Non si può rifiutare l'ipotesi nulla: i dati possono essere considerati normalmente distribuiti.")
else:
    print("Rifiutiamo l'ipotesi nulla: i dati non seguono una distribuzione normale.")

Statistiche K-S: 0.07864280838438042
p-value del test K-S: 0.0121518014474379
Rifiutiamo l'ipotesi nulla: i dati non seguono una distribuzione normale.


In [39]:
t_stat, p_value = ttest_rel(full_differences, np.zeros(len(full_differences)))

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Rifiutiamo l'ipotesi nulla: C'è evidenza statistica di una differenza significativa nelle prestazioni tra i due metodi di interpolazione.")
else:
    print("Non si può rifiutare l'ipotesi nulla: on ci sono evidenze statistiche sufficienti per affermare che ci sia una differenza nelle prestazioni tra i due metodi di interpolazione")

T-statistic: -11.4890
P-value: 0.0000
Rifiutiamo l'ipotesi nulla: C'è evidenza statistica di una differenza significativa nelle prestazioni tra i due metodi di interpolazione.
